In [1]:
import os

import numpy as np
import torch
import torchvision.datasets as dset
import torch.nn as nn
import torchvision.transforms as transforms

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [2]:
#assert pyro.__version__.startswith('1.3.0')
pyro.enable_validation(True)
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run the notebook cells on CI.
#smoke_test = 'CI' in os.environ

In [3]:
def setup_data_loaders(batch_size=128, use_cuda=False):
    root = './data'
    download = True
    trans = transforms.ToTensor()
    train_set = dset.MNIST(root=root, train=True, transform=trans,
                           download=download)
    test_set = dset.MNIST(root=root, train=False, transform=trans)

    kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
    train_loader = torch.utils.data.DataLoader(dataset=train_set,
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset=test_set,
        batch_size=batch_size, shuffle=False, **kwargs)
    return train_loader, test_loader

In [4]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, 784)
        # setup the non-linearities
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x 784
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img

In [5]:
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        # setup the three linear transformations used
        self.fc1 = nn.Linear(784, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the image x
        # first shape the mini-batch to have pixels in the rightmost dimension
        x = x.reshape(-1, 784)
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale

In [6]:
# define the model p(x|z)p(z)
def model(self, x):
    # register PyTorch module `decoder` with Pyro
    pyro.module("decoder", self.decoder)
    with pyro.plate("data", x.shape[0]):
        # setup hyperparameters for prior p(z)
        z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
        # z loc torch.Size([256, 50])
        z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
        # sample from prior (value will be sampled by guide when computing the ELBO)
        z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
        # we sample a Z from a (0, I) normal distribution
        # then we pass it though a nn
        # mu = nn(z)
        # then this mu is used in another dist
        # p(x|z) where z is samples
        # then we sample an x from this
        # the idea is, this nn function learns a distribution
        # that is, what would it be like to sample z from P(z|X)
        # 
        # z shape torch.Size([256, 50])
        # decode the latent code z
        loc_img = self.decoder.forward(z)
        #loc img torch.Size([256, 784])
        # score against actual images
        # bern shape Independent(Bernoulli(probs: torch.Size([256, 784])), 1)
        # 784 is the batch size
        # 256 is the image size
        pyro.sample("obs", dist.Bernoulli(loc_img).to_event(1), obs=x.reshape(-1, 784))

In [7]:
# define the guide (i.e. variational distribution) q(z|x)
def guide(self, x):
    # register PyTorch module `encoder` with Pyro
    pyro.module("encoder", self.encoder)
    with pyro.plate("data", x.shape[0]):
        # use the encoder to get the parameters used to define q(z|x)
        z_loc, z_scale = self.encoder.forward(x)
        # p(z,b) = q(b)mult(i=1 to i=N)q(zi|f(xi))
        
        # given an image, we output a distribution for z
        # then we sample a z. because the guide always gives the
        # approximate posterior, the variational inference
        # sample the latent code z
        pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

In [8]:
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=400, use_cuda=False):
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim)
        self.decoder = Decoder(z_dim, hidden_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    
    
    def model(self, x):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            # setup hyperparameters for prior p(z)
            z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
            z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
            # sample from prior (value will be sampled by guide when computing the ELBO)
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            # decode the latent code z
            loc_img = self.decoder.forward(z)
            # score against actual images
            # decoder is where the image goes 
            pyro.sample("obs", dist.Bernoulli(loc_img).to_event(1), obs=x.reshape(-1, 784))

    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder.forward(x)
            # sample the latent code z
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

    # define a helper function for reconstructing images
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

In [9]:
vae = VAE()

optimizer = Adam({"lr": 1.0e-3})

svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

def train(svi, train_loader, use_cuda=False):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, _ in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x)

    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

def evaluate(svi, test_loader, use_cuda=False):
    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    for x, _ in test_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x)
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test

In [10]:
LEARNING_RATE = 1.0e-3
USE_CUDA = False
smoke_test = False

# Run only for a single iteration for testing
NUM_EPOCHS = 100 if smoke_test else 100
TEST_FREQUENCY = 5
train_loader, test_loader = setup_data_loaders(batch_size=256, use_cuda=USE_CUDA)

# clear param store
pyro.clear_param_store()

# setup the VAE
vae = VAE(use_cuda=USE_CUDA)

# setup the optimizer
adam_args = {"lr": LEARNING_RATE}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))

[epoch 000]  average training loss: 191.0216
[epoch 000] average test loss: 156.0872
[epoch 001]  average training loss: 146.8141
[epoch 002]  average training loss: 133.2540
[epoch 003]  average training loss: 124.6775
[epoch 004]  average training loss: 119.5152
[epoch 005]  average training loss: 116.1240
[epoch 005] average test loss: 113.7908
[epoch 006]  average training loss: 113.7285
[epoch 007]  average training loss: 112.0445
[epoch 008]  average training loss: 110.7292
[epoch 009]  average training loss: 109.7455
[epoch 010]  average training loss: 108.9070
[epoch 010] average test loss: 107.7720
[epoch 011]  average training loss: 108.2513
[epoch 012]  average training loss: 107.6953
[epoch 013]  average training loss: 107.2849
[epoch 014]  average training loss: 106.8870
[epoch 015]  average training loss: 106.4983
[epoch 015] average test loss: 105.9786
[epoch 016]  average training loss: 106.1872
[epoch 017]  average training loss: 105.9363
[epoch 018]  average training 

In [16]:
from pyro.nn import PyroSample, PyroModule
from pyro.distributions import Normal, Categorical

class ClassifierBnn(PyroModule):
    
    def __init__(self, num_in = 100, num_hidden = 200, num_out = 10, prior_std = 1.):
        
        # call to father constructor
        super().__init__()
        
        # define prior
        prior = Normal(0, prior_std)
        
        # Define layers
        
        # linear layer 1
        self.linear_layer = PyroModule[torch.nn.Linear](num_in, num_hidden)
        
        # linear alyer parameters as random variables
        self.linear_layer.weights = PyroSample(prior.expand([num_hidden, num_in]).to_event(2))
        self.linear_layer.bias = PyroSample(prior.expand([num_hidden]).to_event(1))
        
        # linear layer 2
        # output dimension is 3 because of the number of classes
        self.output_layer = PyroModule[torch.nn.Linear](num_hidden, num_out)
        
        # linear alyer parameters as random variables
        self.output_layer.weights = PyroSample(prior.expand([num_out, num_hidden]).to_event(2))
        self.output_layer.bias = PyroSample(prior.expand([num_out]).to_event(1))
        
        # activation function
        #self.activation = torch.nn.functional.softmax()
        
    def forward(self, x, y = None):
            
        # latent variable
        z = self.linear_layer(x)
        z = self.output_layer(z)
        z = torch.nn.functional.log_softmax(z, dim=1)
        # likelihood
        with pyro.plate("data",size = x.shape[0], dim = -1):
            # I think this means each batch is independent            
            # z is the input to the distribution (categorical)
            obs = pyro.sample("obs", Categorical(logits = z), obs=y)
        # return latent variable
        return z

In [17]:
# validate NN

pyro.enable_validation(True)

model = ClassifierBnn()
x, y = next(iter(train_loader))
z_loc, z_scale = vae.encoder(x)
combined_z = torch.cat((z_loc, z_scale), 1)


print(pyro.poutine.trace(model).get_trace(combined_z, y).format_shapes())

         Trace Shapes:            
          Param Sites:            
   linear_layer.weight  10 100    
   output_layer.weight  10  10    
         Sample Sites:            
linear_layer.bias dist       | 200
                 value       | 200
output_layer.bias dist       |  10
                 value       |  10
             data dist       |    
                 value 256   |    
              obs dist 256   |    
                 value 256   |    


In [18]:
pyro.enable_validation(True)
pyro.clear_param_store()
model = ClassifierBnn(num_hidden = 10, prior_std = 1.)

# define guide
from pyro.infer.autoguide import AutoDiagonalNormal
guide = AutoDiagonalNormal(model, init_scale=1e-1)

# define SVI (model for training)
svi = pyro.infer.SVI(model,
                    guide,
                    optim=pyro.optim.ClippedAdam({'lr':1e-2}),
                    # Define conventional ELBO
                     loss=pyro.infer.Trace_ELBO())

In [19]:
from pyro.infer import Predictive
predictive = Predictive(model, guide=guide, num_samples=20)

def predict(x):
    # for a single image, output a mean and sd for category
    yhats = predictive(x)["obs"].double()
    # yhats[0] seems to be integers 0 to 9, len 256
    # prediction for one model, for all items in batch
    # 20, 256
    mean = torch.mean(yhats, axis=0)
    std = torch.std(yhats.float(), 0).numpy()
    # yhats outputs a batch size number of predictions for 20 models
    # yhats seem to be a dictionary of weights
    return mean, std

In [20]:
# Training

num_epochs = 1000

# Define number of epochs
epoch_loss = np.zeros(shape=(num_epochs,))

# training
for epoch in range(10):
    i = 0
    for x, y in train_loader:
        # batches of size 256 are being fed in 
        z_loc, z_scale = vae.encoder(x)
        combined_z = torch.cat((z_loc, z_scale), 1)
        loss = svi.step(combined_z, y)
        mean, std = predict(combined_z)
        accuracy_per_batch = torch.sum(torch.sub(mean,y))/len(y)
        print("loss", loss)
        print("mean", mean[0], "y is", y[0])
        print(accuracy_per_batch)

loss 637.1355237960815
mean tensor(4.2000, dtype=torch.float64) y is tensor(5)
tensor(0.3443, dtype=torch.float64)
loss 626.6906805038452
mean tensor(5.3000, dtype=torch.float64) y is tensor(0)
tensor(0.3221, dtype=torch.float64)
loss 607.4282379150391
mean tensor(4.6500, dtype=torch.float64) y is tensor(4)
tensor(-0.0379, dtype=torch.float64)
loss 588.279972076416
mean tensor(4.1500, dtype=torch.float64) y is tensor(1)
tensor(-0.1799, dtype=torch.float64)
loss 580.9760131835938
mean tensor(4.6000, dtype=torch.float64) y is tensor(0)
tensor(-0.1559, dtype=torch.float64)
loss 576.6481323242188
mean tensor(3.2000, dtype=torch.float64) y is tensor(3)
tensor(-0.3453, dtype=torch.float64)
loss 557.9143877029419
mean tensor(4.5000, dtype=torch.float64) y is tensor(3)
tensor(-0.2748, dtype=torch.float64)
loss 555.7110233306885
mean tensor(4.9500, dtype=torch.float64) y is tensor(4)
tensor(0.2373, dtype=torch.float64)
loss 536.5982828140259
mean tensor(5., dtype=torch.float64) y is tensor(3)
t

loss 144.5504961013794
mean tensor(3.1000, dtype=torch.float64) y is tensor(3)
tensor(0.1932, dtype=torch.float64)
loss 150.53664875030518
mean tensor(3.2500, dtype=torch.float64) y is tensor(3)
tensor(0.0191, dtype=torch.float64)
loss 126.39814686775208
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.0117, dtype=torch.float64)
loss 149.61079025268555
mean tensor(2.9500, dtype=torch.float64) y is tensor(3)
tensor(-0.0482, dtype=torch.float64)
loss 102.30500411987305
mean tensor(2.5000, dtype=torch.float64) y is tensor(3)
tensor(0.0664, dtype=torch.float64)
loss 127.4996976852417
mean tensor(4.4000, dtype=torch.float64) y is tensor(5)
tensor(0.0199, dtype=torch.float64)
loss 130.95302963256836
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.0143, dtype=torch.float64)
loss 140.86411476135254
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(-0.0340, dtype=torch.float64)
loss 123.00174140930176
mean tensor(8., dtype=torch.float64) y is tensor(8)
tensor

loss 109.10559511184692
mean tensor(2.0500, dtype=torch.float64) y is tensor(2)
tensor(0.0248, dtype=torch.float64)
loss 123.87192487716675
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.0682, dtype=torch.float64)
loss 97.45253252983093
mean tensor(1.1000, dtype=torch.float64) y is tensor(1)
tensor(-0.1191, dtype=torch.float64)
loss 113.89771890640259
mean tensor(8., dtype=torch.float64) y is tensor(8)
tensor(0.0318, dtype=torch.float64)
loss 117.80908286571503
mean tensor(5.2500, dtype=torch.float64) y is tensor(5)
tensor(-0.0557, dtype=torch.float64)
loss 110.97034645080566
mean tensor(5.7500, dtype=torch.float64) y is tensor(6)
tensor(-0.0205, dtype=torch.float64)
loss 102.0224871635437
mean tensor(5., dtype=torch.float64) y is tensor(5)
tensor(-0.0574, dtype=torch.float64)
loss 120.07359743118286
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(0.0018, dtype=torch.float64)
loss 129.5422134399414
mean tensor(8.6500, dtype=torch.float64) y is tensor(9)
tensor

loss 116.36570739746094
mean tensor(1.5500, dtype=torch.float64) y is tensor(0)
tensor(-0.1789, dtype=torch.float64)
loss 135.39495873451233
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.1494, dtype=torch.float64)
loss 99.11452007293701
mean tensor(8., dtype=torch.float64) y is tensor(9)
tensor(-0.0561, dtype=torch.float64)
loss 71.2914924621582
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(0.0143, dtype=torch.float64)
loss 94.58977127075195
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(-0.2006, dtype=torch.float64)
loss 133.71141815185547
mean tensor(5.3500, dtype=torch.float64) y is tensor(5)
tensor(-0.2326, dtype=torch.float64)
loss 100.2219467163086
mean tensor(0., dtype=torch.float64) y is tensor(0)
tensor(-0.0359, dtype=torch.float64)
loss 96.87181681394577
mean tensor(7.2000, dtype=torch.float64) y is tensor(8)
tensor(-0.1805, dtype=torch.float64)
loss 105.25891733169556
mean tensor(5.4500, dtype=torch.float64) y is tensor(5)
tensor(-0.1

loss 120.09221839904785
mean tensor(4.1000, dtype=torch.float64) y is tensor(5)
tensor(-0.0578, dtype=torch.float64)
loss 106.0954122543335
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(-0.1184, dtype=torch.float64)
loss 97.4101185798645
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(-0.1453, dtype=torch.float64)
loss 95.63082027435303
mean tensor(8., dtype=torch.float64) y is tensor(8)
tensor(-0.0119, dtype=torch.float64)
loss 106.55313777923584
mean tensor(2.0500, dtype=torch.float64) y is tensor(2)
tensor(-0.0688, dtype=torch.float64)
loss 97.9421854019165
mean tensor(0.2000, dtype=torch.float64) y is tensor(0)
tensor(0.0965, dtype=torch.float64)
loss 89.7940902709961
mean tensor(7.8500, dtype=torch.float64) y is tensor(8)
tensor(-0.0293, dtype=torch.float64)
loss 116.46881294250488
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(0.0814, dtype=torch.float64)
loss 100.68982172012329
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(0.1453,

...

loss 111.80037593841553
mean tensor(2., dtype=torch.float64) y is tensor(2)
tensor(0.0615, dtype=torch.float64)
loss 78.57745361328125
mean tensor(4., dtype=torch.float64) y is tensor(4)
tensor(0.0633, dtype=torch.float64)
loss 89.80871677398682
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(0.0074, dtype=torch.float64)
loss 85.62249851226807
mean tensor(3., dtype=torch.float64) y is tensor(3)
tensor(-0.1143, dtype=torch.float64)
loss 118.93721199035645
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.0957, dtype=torch.float64)
loss 95.19253253936768
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(0.1447, dtype=torch.float64)
loss 103.21644115447998
mean tensor(2., dtype=torch.float64) y is tensor(2)
tensor(0.0533, dtype=torch.float64)
loss 72.72949886322021
mean tensor(6., dtype=torch.float64) y is tensor(6)
tensor(-0.0172, dtype=torch.float64)
loss 79.75090789794922
mean tensor(0., dtype=torch.float64) y is tensor(0)
tensor(0.0068, dtype=torch.floa

loss 73.50930213928223
mean tensor(6., dtype=torch.float64) y is tensor(6)
tensor(-0.0961, dtype=torch.float64)
loss 81.6762228012085
mean tensor(0., dtype=torch.float64) y is tensor(0)
tensor(0.0775, dtype=torch.float64)
loss 95.76903629302979
mean tensor(3.9500, dtype=torch.float64) y is tensor(3)
tensor(-0.0521, dtype=torch.float64)
loss 69.15899562835693
mean tensor(7.1000, dtype=torch.float64) y is tensor(7)
tensor(0.0555, dtype=torch.float64)
loss 90.80842113494873
mean tensor(7., dtype=torch.float64) y is tensor(8)
tensor(0.0400, dtype=torch.float64)
loss 108.1658067703247
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(-0.0385, dtype=torch.float64)
loss 83.1600923538208
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(0.0789, dtype=torch.float64)
loss 64.94384670257568
mean tensor(6., dtype=torch.float64) y is tensor(6)
tensor(-0.0256, dtype=torch.float64)
loss 63.856282234191895
mean tensor(3.2000, dtype=torch.float64) y is tensor(4)
tensor(-0.0570, dtype=

loss 85.97941303253174
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(-0.0406, dtype=torch.float64)
loss 68.61635398864746
mean tensor(5., dtype=torch.float64) y is tensor(5)
tensor(0.0238, dtype=torch.float64)
loss 53.06521797180176
mean tensor(0., dtype=torch.float64) y is tensor(0)
tensor(0.0994, dtype=torch.float64)
loss 117.80403995513916
mean tensor(8.1000, dtype=torch.float64) y is tensor(8)
tensor(0.1156, dtype=torch.float64)
loss 92.03231239318848
mean tensor(3., dtype=torch.float64) y is tensor(3)
tensor(0.1037, dtype=torch.float64)
loss 77.82761096954346
mean tensor(6., dtype=torch.float64) y is tensor(6)
tensor(0.0350, dtype=torch.float64)
loss 107.60990238189697
mean tensor(5., dtype=torch.float64) y is tensor(5)
tensor(0.0473, dtype=torch.float64)
loss 78.09743404388428
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(0.0938, dtype=torch.float64)
loss 90.17822170257568
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(0.1516, dtype=torch.flo

loss 96.48444938659668
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(0.0232, dtype=torch.float64)
loss 97.61210298538208
mean tensor(5.9500, dtype=torch.float64) y is tensor(6)
tensor(-0.0225, dtype=torch.float64)
loss 71.82934761047363
mean tensor(1., dtype=torch.float64) y is tensor(1)
tensor(0.0137, dtype=torch.float64)
loss 70.43137454986572
mean tensor(5.2000, dtype=torch.float64) y is tensor(5)
tensor(-0.0270, dtype=torch.float64)
loss 82.36676979064941
mean tensor(7., dtype=torch.float64) y is tensor(7)
tensor(-0.0922, dtype=torch.float64)
loss 75.36786413192749
mean tensor(3.1500, dtype=torch.float64) y is tensor(3)
tensor(-0.0217, dtype=torch.float64)
loss 67.90523338317871
mean tensor(6.8500, dtype=torch.float64) y is tensor(8)
tensor(-0.0855, dtype=torch.float64)
loss 67.99691772460938
mean tensor(0., dtype=torch.float64) y is tensor(0)
tensor(-0.0592, dtype=torch.float64)
loss 86.08366775512695
mean tensor(4., dtype=torch.float64) y is tensor(4)
tensor(-0.0568,

loss 80.9532585144043
mean tensor(9., dtype=torch.float64) y is tensor(9)
tensor(-0.0910, dtype=torch.float64)
loss 90.22877025604248
mean tensor(3., dtype=torch.float64) y is tensor(9)
tensor(0.0275, dtype=torch.float64)
loss 69.94841384887695
mean tensor(7., dtype=torch.float64) y is tensor(1)
tensor(0.0816, dtype=torch.float64)
loss 70.73832988739014
mean tensor(3.1000, dtype=torch.float64) y is tensor(3)
tensor(-0.0725, dtype=torch.float64)
loss 81.4617280960083
mean tensor(6., dtype=torch.float64) y is tensor(6)
tensor(-0.0191, dtype=torch.float64)
loss 77.6117115020752
mean tensor(4.7500, dtype=torch.float64) y is tensor(5)
tensor(-0.2191, dtype=torch.float64)
loss 79.60945987701416
mean tensor(4.7500, dtype=torch.float64) y is tensor(5)
tensor(-0.1225, dtype=torch.float64)
loss 70.95444011688232
mean tensor(8.1500, dtype=torch.float64) y is tensor(9)
tensor(-0.1457, dtype=torch.float64)
loss 75.4791145324707
mean tensor(7.1000, dtype=torch.float64) y is tensor(7)
tensor(0.0184, 